# EXPLANATION

This is to get sample tweets for us to take a look and start labeling

In [ ]:
import os
from pathlib import Path
import sys

# --- REPO ROOT ON sys.path (so `from src.*` works locally) ---
_REPO_ROOT = str(Path(os.getcwd()).resolve().parents[1])
if _REPO_ROOT not in sys.path:
    sys.path.insert(0, _REPO_ROOT)


# --- ENVIRONMENT SWITCH ---
# Set to True if running on local machine with Google Drive Desktop mounted
# Set to False if running in Google Colab cloud
RUNNING_LOCALLY = True

if RUNNING_LOCALLY:
    # Standard macOS path for Google Drive Desktop
    BASE_PATH = Path('/Volumes/GoogleDrive/MyDrive/AI Public Trust')
else:
    # Google Colab cloud path
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = Path('/content/drive/MyDrive/AI Public Trust')

# Pre-compute critical paths used across notebooks
twits_folder = BASE_PATH / 'Raw Data/Twits/'
test_folder = BASE_PATH / 'Raw Data/'
datasets_folder = BASE_PATH / 'Data Sets'
cleanedds_folder = BASE_PATH / 'Data Sets/Cleaned Data'
networks_folder = BASE_PATH / 'Data Sets/Networks/'
literature_folder = BASE_PATH / 'Literature/'
topic_models_folder = BASE_PATH / 'Models/Topic Modeling/'


# Setup

In [1]:
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from collections import Counter
import json
import time
import os
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
import random
import networkx as nx
import re
import string
import csv

from pathlib import Path
# from google.colab import drive
# drive.mount('/content/drive')

# %%
# ──────────────────────────────────────────────────────────────────────────────
# 1. Drive Mount & Paths
# ──────────────────────────────────────────────────────────────────────────────
# drive.mount('/content/drive')

# Base project folder (Ignacio standard)
BASE = Path('/content/drive/My Drive/Colab Projects/AI Public Trust')


twits_folder = BASE / 'Raw Data/Twits/'
test_folder = BASE / 'Raw Data/'
print("Current Directory:", twits_folder)
datasets_folder = BASE / 'Data Sets'
cleanedds_folder = BASE / 'Data Sets' / 'Cleaned Data'


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current Directory: /content/drive/My Drive/Colab Projects/AI Public Trust/Raw Data/Twits


# Sanity Check

In [2]:
# This is the processing function that the authors of the classifier provide.
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
# Check how it looks, LINE BY LINE
# CHECKING FOR TYPES OF TWEET THAT ARE NOT ORIGINAL
AItrust_twits_dict_test = open(cleanedds_folder/'AItrust_pruned_twits_with_sentiment.json','r',encoding='utf-8')

i = 0
for line in AItrust_twits_dict_test:
    twit = json.loads(line)
    #print(twit)
    try:
      i+=1
      print(twit.keys())
      print(twit['sentiment_label'])
      print(twit['public_metrics']['like_count'])
      print(twit['public_metrics']['retweet_count'])
      print(twit['type'])
      print(twit.keys())
      print('This is the original tweet:')
      print(twit['text'])
      print('\n')
      print('This is the clean tweet:')
      print(preprocess(twit['text']))
      print(twit['created_at'])
      print('------')
    except:
      pass
    #print('------')
    if i>2:
        break
AItrust_twits_dict_test.close()

dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'conversation_id', 'processed_text', 'sentiment_label', 'sentiment_score'])
positive
1
0
original
dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'conversation_id', 'processed_text', 'sentiment_label', 'sentiment_score'])
This is the original tweet:
Hey 
you need perfect scores and completely AI proof content in your assignments and essays🤩🤩?
We are your best made decision this semester we handle:
✏MATLAB
✏Mathematics
✏Coding
✏Accounting/Finance
✏Web
✏Python
✏Tableau, Excel, SPSS Statistics


This is the clean tweet:
Hey 
you need perfect scores and completely AI proof content in your assignments and essays🤩🤩?
We are your best made decision this semester we handle:
✏MATLAB
✏Mathematics
✏Coding
✏Accounting/Finance
✏Web
✏Python
✏Tableau, Excel, SPSS Statistics
2023-02-06T16:59:59.000Z
------
dict_keys(['id', 'text', 'created_at', 'public_metrics', 'author_id', 'type', 'referenced_t

# Extracting Top Tweets

In [6]:
# --- Configuration ---
# Define the number of top tweets you want
TOP_N = 150
# Total number of lines in the file for an accurate progress bar
TOTAL_TWEETS = 17410035

# Define your file paths using pathlib for better compatibility
# Assumes 'cleanedds_folder' is a Path object or a string to your directory
# If 'cleanedds_folder' isn't defined yet, uncomment and set the line below
# cleanedds_folder = Path('path/to/your/data_folder')

input_filepath = cleanedds_folder / 'AItrust_pruned_twits_with_sentiment.json'
output_filepath = cleanedds_folder / f'top_{TOP_N}_tweets_by_likes_with_id.csv'

# --- Main Logic ---
all_tweets = []

try:
    with open(input_filepath, 'r', encoding='utf-8') as f:
        print(f"🚀 Reading {TOTAL_TWEETS:,} tweets from {input_filepath}...")
        progress_bar = tqdm(f, total=TOTAL_TWEETS, desc="Processing tweets", unit=" tweets")

        for line in progress_bar:
            try:
                tweet = json.loads(line)
                # Extract the necessary information, NOW INCLUDING THE TWEET ID
                extracted_info = {
                    'id': tweet.get('id'), # <-- ADDED THIS LINE
                    'text': tweet.get('text'),
                    'likes': tweet.get('public_metrics', {}).get('like_count', 0),
                    'retweets': tweet.get('public_metrics', {}).get('retweet_count', 0),
                    'sentiment': tweet.get('sentiment_label')
                }
                all_tweets.append(extracted_info)
            except (json.JSONDecodeError, KeyError):
                pass

except FileNotFoundError:
    print(f"❌ ERROR: The input file was not found at {input_filepath}")
    exit()

print(f"\n✅ Found and processed {len(all_tweets)} tweets.")

print("Sorting tweets by like count...")
sorted_tweets = sorted(all_tweets, key=lambda x: x['likes'], reverse=True)

top_tweets = sorted_tweets[:TOP_N]

print(f"Writing the top {len(top_tweets)} tweets to {output_filepath} using pandas...")

# --- Write to CSV using pandas ---
try:
    # 1. Convert the list of dictionaries into a pandas DataFrame
    df = pd.DataFrame(top_tweets)

    # 2. (Optional but good practice) Define and set the column order
    column_order = ['id', 'text', 'likes', 'retweets', 'sentiment']
    df = df[column_order]

    # 3. Use the DataFrame's .to_csv() method to save the file
    df.to_csv(output_filepath, index=False, encoding='utf-8')

    print("🎉 Successfully created the CSV file!")

except Exception as e:
    print(f"❌ ERROR: Could not write to the file. Pandas error: {e}")

🚀 Reading 17,410,035 tweets from /content/drive/My Drive/Colab Projects/AI Public Trust/Data Sets/Cleaned Data/AItrust_pruned_twits_with_sentiment.json...


Processing tweets: 100%|██████████| 17410035/17410035 [04:53<00:00, 59347.84 tweets/s]



✅ Found and processed 17410035 tweets.
Sorting tweets by like count...
Writing the top 150 tweets to /content/drive/My Drive/Colab Projects/AI Public Trust/Data Sets/Cleaned Data/top_150_tweets_by_likes_with_id.csv using pandas...
🎉 Successfully created the CSV file!
